# Pokemon selection

After obtaining the predictions for the battle results of the available Pokemon against the enemy team, we will now select the optimal set of those Pokemon that performs best against the enemies using Mixed Linear Integer programming.

## 0. Environmental setup

In [5]:
from __future__ import print_function
import ortools
import pandas as pd
import numpy as np


from ortools.linear_solver import pywraplp

---

## 1. Read in data

We will now read in the predictions for the battle results obtained from our deep model.

In [6]:
battle_preds = pd.read_csv('battle_preds.csv')

In [7]:
battle_preds.head()

,Name_1,Level_1,Price_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,...,res_krabby,res_mewtwo,res_raichu,res_venusaur,hppr_caterpie,hppr_golem,hppr_krabby,hppr_mewtwo,hppr_raichu,hppr_venusaur
0,Abra,4,346,32,22,17,115,61,99,False,...,-179.35338,-684.09970,-222.42577,-173.653670,0.000000,0.0,0.0,0.0,0.0,0.0
1,Abra,26,614,80,38,26,197,103,170,False,...,-178.42506,-679.82420,-221.97230,-157.285130,0.887533,0.0,0.0,0.0,0.0,0.0
2,Abra,33,775,102,48,33,247,131,214,False,...,-155.15506,-679.60570,-219.45630,-99.916725,0.925494,0.0,0.0,0.0,0.0,0.0
3,Abra,45,937,129,56,39,298,158,257,False,...,-148.49413,-681.31476,-217.22935,-52.117302,0.972685,0.0,0.0,0.0,0.0,0.0
4,Abra,46,963,133,58,40,306,162,264,False,...,-147.37520,-681.24970,-216.34810,-36.872240,0.976467,0.0,0.0,0.0,0.0,0.0


In [8]:
battle_outcomes = battle_preds[['hppr_caterpie', 'hppr_golem', 'hppr_krabby', 'hppr_mewtwo', 'hppr_raichu', 'hppr_venusaur']]
battle_outcomes = np.array(battle_outcomes)

num_enemy_pokemon = 6
num_available_pokemon = len(battle_outcomes)

prices = battle_preds['Price_1']
prices = np.array(prices)

max_budget = 3500

In [9]:
np.mean(battle_outcomes, axis=0)

array([0.96058102, 0.32479792, 0.33058481, 0.02904102, 0.38795036,
       0.55356258])

---

## 2. Mixed Integer Programming

First, we crate the solver that can be used to solve the program.

In [10]:
# Create the mip solver with the CBC backend.
solver = pywraplp.Solver('pokemon_mip_problem',
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

Second we define the variables of our program.

In [11]:
x = {}

for i in range(num_available_pokemon):
    for j in range(num_enemy_pokemon):
        x[i, j] = solver.BoolVar('x[%i,%i]' % (i, j))


Third, we define the objective function. To this end we first need to define the rewards matrix, that is the remaining HP% after battling. Those information we get from our prediction model.

In [12]:
solver.Maximize(solver.Sum([battle_outcomes[i][j] * x[i,j] for i in range(num_available_pokemon)
                                                  for j in range(num_enemy_pokemon)]))

Fourth, we define the constraints of our problem.

In [13]:
 # Each available pokemon is assigned to at most 1 enemy pokemon.

for i in range(num_available_pokemon):
    solver.Add(solver.Sum([x[i, j] for j in range(num_enemy_pokemon)]) <= 1)

# Each enemy pokemon gets assigned to exactly one available pokemon.

for j in range(num_enemy_pokemon):
    solver.Add(solver.Sum([x[i, j] for i in range(num_available_pokemon)]) == 1)

# The price of the selected pokemon must be at most equal to the budget
solver.Add(solver.Sum([x[i, j] * prices[i] for i in range(num_available_pokemon) for j in range(num_enemy_pokemon)]) <= max_budget)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f7a265e5db0> >

We finally invoke the solver.

In [14]:
  sol = solver.Solve()

  print('Total HPPR = ', solver.Objective().Value())
  print()
  for i in range(num_available_pokemon):
    for j in range(num_enemy_pokemon):
      if x[i, j].solution_value() > 0:
        print('Pokemon %d assigned to enemy %d.  HPPR = %f   Price = %d' % (
              i,
              j,
              battle_outcomes[i][j], prices[i]))

  print()
  print("Time = ", solver.WallTime(), " milliseconds")

Total HPPR =  3.1290258842784895

Pokemon 200 assigned to enemy 2.  HPPR = 0.000000   Price = 265
Pokemon 208 assigned to enemy 4.  HPPR = 0.945848   Price = 1042
Pokemon 610 assigned to enemy 3.  HPPR = 0.000000   Price = 205
Pokemon 970 assigned to enemy 0.  HPPR = 0.805214   Price = 288
Pokemon 1125 assigned to enemy 1.  HPPR = 0.448289   Price = 956
Pokemon 1222 assigned to enemy 5.  HPPR = 0.929675   Price = 738

Time =  3171  milliseconds
